In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install transformers datasets evaluate accelerate

In [ ]:
from datasets import load_dataset

mental = load_dataset("Kanakmi/mental-disorders")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/465051 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23190 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/92976 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_mental = mental.map(preprocess_function, batched=True)

Map:   0%|          | 0/465051 [00:00<?, ? examples/s]

Map:   0%|          | 0/23190 [00:00<?, ? examples/s]

Map:   0%|          | 0/92976 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: 'BPD', 1:'bipolar', 2:'depression', 3:'anxiety', 4:'schizophrenia', 5:'mentalillness'}
label2id = {'BPD': 0, 'bipolar':1, 'depression':2, 'anxiety':3, 'schizophrenia':4, 'mentalillness':5}

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=[
"q_lin",
"k_lin",
"v_lin",
"out_lin"])

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 890,118 || all params: 67,848,204 || trainable%: 1.3119256627633062


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="dbrill1421/distilbert_mental_lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_mental["train"],
    eval_dataset=tokenized_mental["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.004700,1.007953,0.644329
2,0.925800,0.919362,0.667486


TrainOutput(global_step=29066, training_loss=0.9439690451509469, metrics={'train_runtime': 36286.4097, 'train_samples_per_second': 25.632, 'train_steps_per_second': 0.801, 'total_flos': 1.246196240321426e+17, 'train_loss': 0.9439690451509469, 'epoch': 2.0})

In [ ]:
model.save_pretrained("output_dir")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()
model.push_to_hub("dbrill1421/distilbert_mental_lora")

README.md:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/271M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dbrill1421/distilbert_mental_lora/commit/afd809140a09e5c24979f9672b19e7f16ff0c0f8', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='afd809140a09e5c24979f9672b19e7f16ff0c0f8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "I have health anxiety where I go to the doctor all the time because of a simple headache or acid reflex and I think im having a heart attack or I look stuff up on google and I think I have cancer. I need some tips please I cant live like this."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbrill1421/distilbert_mental_lora")
inputs = tokenizer(text, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import torch

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("dbrill1421/distilbert_mental_lora")
with torch.no_grad():
    logits = model(**inputs).logits

Some weights of the model checkpoint at dbrill1421/distilbert_mental_lora were not used when initializing DistilBertForSequenceClassification: ['classifier.modules_to_save.default.bias', 'classifier.modules_to_save.default.weight', 'classifier.original_module.bias', 'classifier.original_module.weight', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.out_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.out_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.out_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.out_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.weight'

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'